In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [97]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [29]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## Trend実験

In [14]:
# mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset()
df_all, df_census = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=False, coord=False, co_est=True, fix_pop=False, add_location=False, outlier=False)
# df_all = preprocess.add_lag_features(df_all)

/tmp/working/code-analysis/../modules/utils.py:268: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  df_census = load_census(BASE)


In [137]:
low_trend_params = {
    1: {
        'n':3,
        'thre':3,
        'thre_r':0,
        'lower_bound': 60,
        'upper_bound': 140,
        'use_regularize': True,
        'v_regularize': [0.03, 0.02],
        'v_clip':[0.999, 1.004]
    }
}
params = low_trend_params[1]
df_trend, trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(trend_dict))
trend_dict

13


{13025: 1.004,
 13193: 0.999,
 18025: 0.999,
 21147: 1.004,
 31093: 1.0033846255390906,
 40005: 1.004,
 40069: 1.00303696951699,
 46091: 1.0016915440454517,
 48305: 1.004,
 48385: 1.0026579234749724,
 5067: 1.004,
 54023: 0.999,
 8033: 1.004}

In [160]:
high_trend_params = {
    1: {
        'n':3,
        'thre':3,
        'thre_r':0,
        'lower_bound': 20000,
        'upper_bound': 999999,
        'use_regularize': True,
        'v_regularize': [0.01, 0.006],
        'v_clip':[0.999, 1.004]
    }
}
params = high_trend_params[1]
df_trend, high_trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(high_trend_dict))
high_trend_dict

33


{10003: 1.004,
 12031: 1.0025810474983163,
 12081: 1.0021944719741633,
 12101: 1.0004422558912913,
 12127: 0.999,
 17197: 1.0019724841950757,
 24003: 1.0013556096262333,
 24005: 1.004,
 24510: 0.999,
 25025: 1.000131850354836,
 29095: 1.0012930980306953,
 32003: 1.001383320929266,
 34013: 1.0019734470108055,
 36059: 1.000614093499516,
 36085: 1.004,
 37119: 1.000061828055089,
 37183: 1.004,
 39061: 1.004,
 40109: 1.004,
 40143: 1.0022818100881399,
 42003: 1.004,
 47065: 1.0024142195533001,
 47187: 1.0014889726359355,
 48085: 1.004,
 48121: 1.004,
 48157: 1.004,
 48491: 1.001423889702986,
 55133: 1.000885770268916,
 6001: 1.004,
 6065: 1.0008005797236106,
 6073: 1.0039667901607097,
 6087: 1.0036614749940644,
 8005: 1.0014128289889006}

In [157]:
df_submission = pd.read_csv('../submission/submission_13827.csv', index_col='row_id')
df_sub1, df_extract, var_dict = utils.insert_trend(df_submission, df_all, df_census, trend_dict)
df_sub1, df_extract, var_dict = utils.insert_trend(df_sub1, df_all, df_census, high_trend_dict, method='mean')
df_sub1.to_csv('../submission/submission_13827_trend.csv')

In [158]:
df_merged = utils.compare_submission(df_sub1, 'submission_13827')
df_diff = df_merged[df_merged['smape']>0.001].copy()
print(df_diff.shape)

(31, 6)


In [159]:
df_merged['smape'].sum()

5.7174896856607305

In [7]:
# df_submission = pd.read_csv('../submission/submission_2023-02-27_16_35_41.csv', index_col='row_id')
df_submission = pd.read_csv('../submission/submission_2023-02-28_18_21_26.csv', index_col='row_id')
# df_submission = pd.read_csv('../submission/submission_2023-02-27_19_41_33.csv', index_col='row_id')
# df_submission = pd.read_csv('../submission/submission_2023-02-27_21_15_16.csv', index_col='row_id')
df_merged = utils.compare_submission(df_submission, 'submission_13827')
print(df_merged['smape'].mean())

0.016963210799712435


In [235]:
len(df_all['cfips'].unique())

3135

In [ ]:
0.01944239446655972